## 모델 적용 및 평가
1. 로지스틱 회귀 모델
2. XGBoost 

* 하이퍼파라미터 고정
    - test_size=0.2
    - random_state=42 

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, f1_score


In [4]:
# 데이터 로드
df = pd.read_csv('./data/gym_churn_us.csv')
df.columns

Index(['gender', 'Near_Location', 'Partner', 'Promo_friends', 'Phone',
       'Contract_period', 'Group_visits', 'Age',
       'Avg_additional_charges_total', 'Month_to_end_contract', 'Lifetime',
       'Avg_class_frequency_total', 'Avg_class_frequency_current_month',
       'Churn'],
      dtype='object')

In [9]:
df.describe()

,gender,Near_Location,Partner,Promo_friends,Phone,Contract_period,Group_visits,Age,Avg_additional_charges_total,Month_to_end_contract,Lifetime,Avg_class_frequency_total,Avg_class_frequency_current_month,Churn
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,0.510250,0.845250,0.486750,0.308500,0.903500,4.681250,0.412250,29.184250,146.943728,4.322750,3.724750,1.879020,1.767052,0.265250
std,0.499957,0.361711,0.499887,0.461932,0.295313,4.549706,0.492301,3.258367,96.355602,4.191297,3.749267,0.972245,1.052906,0.441521
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,18.000000,0.148205,1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,27.000000,68.868830,1.000000,1.000000,1.180875,0.963003,0.000000
50%,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,29.000000,136.220159,1.000000,3.000000,1.832768,1.719574,0.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,31.000000,210.949625,6.000000,5.000000,2.536078,2.510336,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000,1.000000,41.000000,552.590740,12.000000,31.000000,6.023668,6.146783,1.000000


In [ ]:
# 데이터 분리
X = df.drop('Churn', axis=1)
y = df['Churn']

# 이진으로 분류된 데이터를 제외하고 범주형 데이터를 위한 전처리
columns = ['Avg_additional_charges_total', 'Month_to_end_contract', 'Lifetime', 'Avg_class_frequency_total', 'Avg_class_frequency_current_month']
scaler = StandardScaler()
df[columns] =scaler.fit_transform(df[columns])



# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# # 로지스틱 모델
# lr = LogisticRegression()
# lr.fit(X_train_std, y_train)

# # 평가
# # lr.score(X_train_std, y_train), lr.score(X_test_std, y_test)

In [26]:
X[['Age', 'Month_to_end_contract', 'Lifetime', 'Avg_class_frequency_total', 'Avg_class_frequency_current_month']]



,Age,Month_to_end_contract,Lifetime,Avg_class_frequency_total,Avg_class_frequency_current_month
0,29,5.0,3,0.020398,0.000000
1,31,12.0,7,1.922936,1.910244
2,28,1.0,2,1.859098,1.736502
3,33,12.0,2,3.205633,3.357215
4,26,1.0,3,1.113884,1.120078
...,...,...,...,...,...
3995,33,12.0,8,2.284497,2.349070
3996,29,1.0,1,1.277168,0.292859
3997,28,11.0,2,2.786146,2.831439
3998,32,5.0,3,1.630108,1.596237
